# 0. Initialize

## 0.1. Import Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt

## 0.2. DEFINE VARIABLES 

In [2]:
DATA_PATH = '/Users/Gamegaraj/Desktop/CS412/data/' # '<insert-your-training-data-path-here>'

ROUND = 3 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = '26772'#'<insert-your-id-here>'
PROJECT_CODE = 'CS4129a709ea5dfc4'#'<insert-your-code-here>' # Same code for the annotation eg. CS412xxxxx

## 0.3. Read Training & Evaluation Data

### 0.3.1. Get the labels for tweets

In [3]:
#trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH))
trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
trainingTweetDf

,tweet_id,isPolitical
0,1597170281545551872,Yes
1,1431700027471192069,No
2,1566035577090281472,Yes
3,1591538690869940225,Yes
4,1583898169238167554,Yes
...,...,...
2995,1593539327623151619,Yes
2996,1393886554062524418,No
2997,1597925615092764672,Yes
2998,1585291418616176640,Yes


In [4]:
trainingTweetDf.isPolitical.value_counts()

Yes    2003
No      997
Name: isPolitical, dtype: int64

### 0.3.2. Get the labels for users

In [5]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))
#trainingUserDf = pd.read_csv('training-user.csv')
trainingUserDf

,screen_name,isBot
0,koftecancaddy,No
1,ahaber,No
2,selahat03949652,No
3,erdin06357062,No
4,bhct__necatii,No
...,...,...
2995,djblumenberg,No
2996,mel1sq,No
2997,eren_yz1,Yes
2998,ergnyildiz4,No


In [6]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

### 0.3.3. Expand your dataset with metadata and tweets

In [7]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_CS4129a709ea5dfc4.html


In [8]:
### Add Annotation Data
codes = ['CS4129a709ea5dfc4','CS412a32e72d94b5f','CS412c255f188f1f1',
         'CS4125691a2d1c16d','CS412aa4c69f55b37','CS412c38461f5453a',
        'CS4120462ab3cc7f8']
                                  
def mergeAnnotations(codes):
    df_users = pd.DataFrame()
    for i in codes:
        filename = 'annotated_users_{}.csv'.format(i)
        filename = ('{}'+filename)
        df = pd.read_csv(filename.format(DATA_PATH))
        df = df.drop(['url', 'isOrganizational', 'isTroll','gender'], axis=1)
        df = df.drop(df[df.isBot == 'Not sure'].index)
        df.rename(columns = {'Unnamed: 0':'screen_name'}, inplace = True)
        df.dropna(inplace=True)
        df_users = pd.concat([df,df_users])
    

    df_tweets = pd.DataFrame()
    for i in codes:
        filename = 'annotated_tweets_{}.csv'.format(i)
        filename = ('{}'+filename)
        df = pd.read_csv(filename.format(DATA_PATH))
        df = df.drop(['url', 'sentiment', 'isExperiential','isInsult','topics'], axis=1)
        df.rename(columns = {'Unnamed: 0':'tweet_id'}, inplace = True)
        df.dropna(inplace=True)
        df_tweets = pd.concat([df,df_tweets])  
        df_tweets['tweet_id'] = df_tweets['tweet_id'].astype(str)
    
    return df_users, df_tweets

In [9]:
df_users, df_tweets = mergeAnnotations(codes)

trainingUserDf = pd.concat([trainingUserDf,df_users],ignore_index=True)
trainingTweetDf = pd.concat([trainingTweetDf,df_tweets],ignore_index=True)

# 1. EXTRACT FEATURES
Under *1.1. Political Tweet Detection* and *1.2. Bot Detection*, we firstly collect raw data for processing. We then combine some of them (total_interactions = num_favorites + num_retweets) or use them to extract features (whether the tweet has one of the political entities @meralaksener, @kilicdarogluk etc.).

We expect you to collect more raw data from **tweet_metadata**, **user_profiles** and **user_tweets** files by creating a function as shown in below examples such as *check_if_retweet()* and using it while iterating over data as shown under *Merge Collected Features*.

We also expect you to create new variables as much as you can from the data in order to make your predictions more accurate. For example, you may want to check:

- The tweet sources that a user frequently uses
- Whether the user is a verified account or not

...

to assess whether **a user is a bot or not** and whether **a tweet is political or not**.

In [10]:
PATH_TO_DOWNLOADED = DATA_PATH # 'D:/Users/suuser/Desktop/Sabancı/CS412/spring-2022/project/'

## 1.1. Political Tweet Detection
This part stands for the feature extraction of tweets. We start with collecting the raw data from *tweet_metadata*, then use some of them to extract features.

### 1.1.1. Get Raw Data

#### 1.1.1.1. Check if Retweet

In [11]:
def check_if_retweet(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = ''

    try:
        tweet_metadata_line['retweeted_status']
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name']
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

#### 1.1.1.1. Check if Reply

In [12]:
def check_if_reply(tweet_metadata_line):
    is_reply = 0
    replied_username = ''

    
    if(tweet_metadata_line['in_reply_to_screen_name'] != None ):
        replied_username = tweet_metadata_line['in_reply_to_screen_name']
        is_reply = 1



    return is_reply, replied_username

#### 1.1.1.1. Check if Verified

In [13]:
def check_if_verified(tweet_metadata_line):
     
    is_verified = tweet_metadata_line['user']['verified']  
    return is_verified

#### 1.1.1.2. Get Tweet Text

In [14]:
def get_tweet_text(tweet_metadata_line):
    text = tweet_metadata_line['text']
    
    return text

#### 1.1.1.3. Get Tweet ID

In [15]:
def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

#### 1.1.1.4. Get Number of Mentions and Hashtags

In [16]:
def get_number_mentions_hashtags(tweet_metadata_line):
    names = []
    num_mentions = len(tweet_metadata_line['entities']['user_mentions'])
    if(num_mentions!=0):
        mentions = tweet_metadata_line['entities']['user_mentions']
        for mention in mentions:
            names.append(mention['screen_name'])
    num_hashtags = len(tweet_metadata_line['entities']['hashtags'])
    avg = 0
    if(num_hashtags!=0):
        count = 0
        hashtags = tweet_metadata_line['entities']['hashtags']
        for hashtag in hashtags:
             count += len(hashtag['text'])
        avg = count/len(hashtags)
    return num_mentions, names, num_hashtags, avg

#### 1.1.1.5. Get Number of Retweets and Favorites

In [17]:
def get_number_retweets_favorites(tweet_metadata_line):
    retweet_count = tweet_metadata_line['retweet_count']
    favorite_count = tweet_metadata_line['favorite_count']
    
    return retweet_count, favorite_count

#### 1.1.1.5. Get Number of Punctuations

In [18]:
def get_punc_num(text):
    punc = ['.', ',', '!', '?', ':', ';']
    count = 0
    for x in punc:
        if x in text:
            count += 1
    return count

In [19]:
def nonalpha(string):
  count = 0
  # check each character in the string
  for char in string:
     # increment by 1 if it's non-alphanumeric
     if not char.isalpha():
       count += 1           

  # Take whichever is smaller
  return count


#### 1.1.1.6. Get User Info

In [20]:
def get_user_info(tweet_metadata_line):
    id = tweet_metadata_line['user']['id_str']
    screen_name = tweet_metadata_line['user']['screen_name'].lower()
    description = tweet_metadata_line['user']['description']

    return id, screen_name, description

### 1.1.2. Derive Manually Crafted Features

#### 1.1.2.1. Check for political entity in text

In [21]:
def check_political_ent(text):
    
    # the list below can be modified and some new names may be added (or removed)
    # list_of_entities = ['meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag']
    list_of_entities = ['meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag','vedatbilgn','SavciSayan','fahrettinaltun','erdogan','devlet bahçeli','bahçeli','ülkü'
                        'tcbestepe','fuatoktay','suleymansoylu','ikalin1','ekrem_imamoglu','mansuryavas06','pkk','recep','tayyip','erdoğan','cb','#ak','cumhuriyet'
                        'hdp','terörist','chpkk','kadro','ek','tcbestepe','MevlutCavusoglu', 'ozdag', 'özdağ', 'TBMM','drfahrettinkoca', 'yenisafak', 'tayyip', 'cumhur', 'belediye', 'baskan', 'başkan', 'ulusal',
                        'odatv', 'suleyman', 'haskologlu', 'mansur', 'dbdevletbahceli', 'Ahmet_Davutoglu', 'babacan', 'gazetesozcu', 'imamoglu', 'imamoğlu', 'parlament', 'meclis', 
                        'savaş', 'eğitim', 'egitim', 'dolar', 'lira', 'enflasyon', 'euro', 'döviz', 'altın', 'benzin', 'atama', 'altılı masa', 'abd', 'avrupa', 'almanya', 'nato',
                        'sınır', 'göçmen', 'gocmen', 'sığınmacı', 'mülteci', 'mahkeme', 'kanun', 'ukrayna', 'rusya', 'komisyon', 'fetö', 'faiz', 'piyasa', 'banka', 'politik', 'toplantı', 'çiftçi',
                        'saray', 'demokrasi', 'faşist', 'kemal', 'rejim', 'özgürlük', 'koalisyon', 'egemen', 'kurultay', 'danıştay', 'davutoğlu', 'birleşmiş milletler',
                        'bahçeli', 'diplomasi', 'cem uzan', 'lgbt', 'seçim', '2023', 'ibb', 'cemaat', 'soylu', 'liberal', 'kapital', 'protesto', 'halk','seçim', 
                        'erken seçim','sandık','icraat','gençlik kolu','akp','chp','mhp', 'kayyum','anayasa','mahkeme','nebati','tcmb','merkez bankası','kılışdar','cumhurbaşkanı adayı','millet','zillet'
                        'rte','atama','EYT','atanamıyor','kanun','soruşturma','sorusturma','altılı masa','bakanım','af','genel af', 'skandal','yolsuzluk','ihale','zafer','genelaf','muhalefet','muhalif','bakan'
                       'meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag','vedatbilgn','SavciSayan','fahrettinaltun',
                        'tcbestepe','fuatoktay','suleymansoylu','ikalin1','ekrem_imamoglu','mansuryavas06','murat_kurum',
                       'MevlutCavusoglu', 'drfahrettinkoca', 'NureddinNebati','akaraismailoglu','DIBAliErbas','ismailcatakli',
                    'dbdevletbahceli', 'Ahmet_Davutoglu','deryayanikashb','suleymansoylu','06melihgokcek','mustafasentop','VahitKirisci',
                     'fuatoktay','Akparti','emineerdoğan','iletisim','bybekirbozdag','omerrcelik','kasapoglu']
    entities_in_text = [ent for ent in list_of_entities if ent.lower() in text.lower()]
    number_entities = len(entities_in_text)

    return number_entities

In [22]:
def description_entities(text):
    list_of_entities = ['meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag', 'parti', 'CHP', 'AKP', 'HDP', 'MHP', 'mustafa kemal', 'ekonomi', 'dolar', 'halk', 'afgan', 'suriye', 'islam', 'siyasal', 'terör', 'pkk', 'belediye', 'gençlik', 'fahrettin koca', 'bakanım', 'başkanım', 'T.C.', 'reis', 'demirtaş', 'öcalan', 'asker', 'mehmetçik', 'uzun adam', 'meral akşener', 'kemal kılıçdaroğlu', 'TBMM', 'ittifak', 'koalisyon', 'hükümet', 'devlet', 'bakan', 'dolar', 'euro', 'enflasyon', 'vekil', 'aday', 'seçim', 'kongre', 'meclis', 'miting', 'soylu', 'anayasa', 'mahkeme'
                       ,'meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag','vedatbilgn','SavciSayan','fahrettinaltun','erdogan','devlet bahçeli','bahçeli','ülkü'
                        'tcbestepe','fuatoktay','suleymansoylu','ikalin1','ekrem_imamoglu','mansuryavas06','pkk','recep','tayyip','erdoğan','cb','#ak','cumhuriyet'
                        'hdp','terörist','chpkk','kadro','ek','tcbestepe','MevlutCavusoglu', 'ozdag', 'özdağ', 'TBMM','drfahrettinkoca', 'yenisafak', 'tayyip', 'cumhur', 'belediye', 'baskan', 'başkan', 'ulusal',
                        'odatv', 'suleyman', 'haskologlu', 'mansur', 'dbdevletbahceli', 'Ahmet_Davutoglu', 'babacan', 'gazetesozcu', 'imamoglu', 'imamoğlu', 'parlament', 'meclis', 
                        'savaş', 'eğitim', 'egitim', 'dolar', 'lira', 'enflasyon', 'euro', 'döviz', 'altın', 'benzin', 'atama', 'altılı masa', 'abd', 'avrupa', 'almanya', 'nato',
                        'sınır', 'göçmen', 'gocmen', 'sığınmacı', 'mülteci', 'mahkeme', 'kanun', 'ukrayna', 'rusya', 'komisyon', 'fetö', 'faiz', 'piyasa', 'banka', 'politik', 'toplantı', 'çiftçi',
                        'saray', 'demokrasi', 'faşist', 'kemal', 'rejim', 'özgürlük', 'koalisyon', 'egemen', 'kurultay', 'danıştay', 'davutoğlu', 'birleşmiş milletler',
                        'bahçeli', 'diplomasi', 'cem uzan', 'lgbt', 'seçim', '2023', 'ibb', 'cemaat', 'soylu', 'liberal', 'kapital', 'protesto', 'halk','seçim', 
                        'erken seçim','sandık','icraat','gençlik kolu','akp','chp','mhp', 'kayyum','anayasa','mahkeme','nebati','tcmb','merkez bankası','kılışdar','cumhurbaşkanı adayı','millet','zillet'
                        'rte','atama','EYT','atanamıyor','kanun','soruşturma','sorusturma','altılı masa','bakanım','af','genel af', 'skandal','yolsuzluk','ihale','zafer','genelaf','muhalefet','muhalif','bakan'
                       'meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag','vedatbilgn','SavciSayan','fahrettinaltun',
                        'tcbestepe','fuatoktay','suleymansoylu','ikalin1','ekrem_imamoglu','mansuryavas06','murat_kurum',
                       'MevlutCavusoglu', 'drfahrettinkoca', 'NureddinNebati','akaraismailoglu','DIBAliErbas','ismailcatakli',
                    'dbdevletbahceli', 'Ahmet_Davutoglu','deryayanikashb','suleymansoylu','06melihgokcek','mustafasentop','VahitKirisci',
                     'fuatoktay','Akparti','emineerdoğan','iletisim','bybekirbozdag','omerrcelik','kasapoglu','AK','Parti','seçim','AKP','CHP','MHP','Zafer Partisi']
    entities_in_text = [ent for ent in list_of_entities if ent.lower() in text.lower()]
    number_entities = len(entities_in_text)

    return number_entities

In [23]:
def check_political_mention(ment):
    list_of_entities =['suleymansoylu', 'drfahrettinkoca','ekrem_imamoglu','RT_Erdogan','kilicdarogluk','meral_aksener',
                        'alibabacan','Ahmet_Davutoglu','umitozdag','mansuryavas06',
                         'RTErdogan','omerrcelik','suleymansoylu' ,'EmineErdogan' ,'deryayanikashb' ,'Akparti' ,'omerrcelik',
                       'bybekirbozdag','vedatbilgn','tcmeb','yilmaznazif','ayhan_ogan','ismailcatakli','hasandogan',
                        'osmannnurika','bbismailerdem','meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag','vedatbilgn','SavciSayan','fahrettinaltun',
                        'tcbestepe','fuatoktay','suleymansoylu','ikalin1','ekrem_imamoglu','mansuryavas06','murat_kurum',
                       'MevlutCavusoglu', 'drfahrettinkoca', 'NureddinNebati','akaraismailoglu','DIBAliErbas','ismailcatakli',
                    'dbdevletbahceli', 'Ahmet_Davutoglu','deryayanikashb','suleymansoylu','06melihgokcek','mustafasentop','VahitKirisci',
                     'fuatoktay','Akparti','emineerdoğan','iletisim','bybekirbozdag','omerrcelik','kasapoglu']
    entities_in_text = [ent for ent in list_of_entities if ent in ment]
    return len(entities_in_text)


In [24]:
def check_political_reply(rep):
    list_of_entities = ['suleymansoylu', 'drfahrettinkoca','ekrem_imamoglu','RT_Erdogan','kilicdarogluk','meral_aksener',
                        'alibabacan','Ahmet_Davutoglu','umitozdag','mansuryavas06',
                         'RTErdogan','omerrcelik','suleymansoylu' ,'EmineErdogan' ,'deryayanikashb' ,'Akparti' ,'omerrcelik',
                       'bybekirbozdag','vedatbilgn','tcmeb','yilmaznazif','ayhan_ogan','ismailcatakli','hasandogan',
                        'osmannnurika','bbismailerdem','meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag','vedatbilgn','SavciSayan','fahrettinaltun',
                        'tcbestepe','fuatoktay','suleymansoylu','ikalin1','ekrem_imamoglu','mansuryavas06','murat_kurum',
                       'MevlutCavusoglu', 'drfahrettinkoca', 'NureddinNebati','akaraismailoglu','DIBAliErbas','ismailcatakli',
                    'dbdevletbahceli', 'Ahmet_Davutoglu','deryayanikashb','suleymansoylu','06melihgokcek','mustafasentop','VahitKirisci',
                     'fuatoktay','Akparti','emineerdoğan','iletisim','bybekirbozdag','omerrcelik','kasapoglu']
    
    if(rep in list_of_entities):
        return 1
    else:
        return 0

In [25]:
def get_user_info_metadata_tweet(user_metadata_line):
    user_metadata_line = user_metadata_line['user']
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    user_created_at = user_metadata_line['created_at']
    user_protected = user_metadata_line['protected']
    user_verified = user_metadata_line['verified']
    user_geo_enabled = user_metadata_line['geo_enabled']
    user_tweet_count = user_metadata_line['statuses_count']
    user_fav_count = user_metadata_line['favourites_count']
    
    
    dictionary = { 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count, 
                  'user_created_at':user_created_at, 'user_protected':user_protected,
                   'user_geo_enabled':user_geo_enabled, 
                  'user_tweet_count':user_tweet_count, 'user_fav_count':user_fav_count}

    return dictionary

#### 1.1.2.2. Number of total interactions

In [26]:
def total_interactions(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count
    
    return total_num_interactions

#### 1.2.1.1. Get user info metadata

In [27]:
def get_user_info_metadata(user_metadata_line):
    
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    user_created_at = user_metadata_line['created_at']
    user_protected = user_metadata_line['protected']
    user_verified = user_metadata_line['verified']
    user_geo_enabled = user_metadata_line['geo_enabled']
    user_tweet_count = user_metadata_line['statuses_count']
    user_fav_count = user_metadata_line['favourites_count']
    
    
    dictionary = {'user_id':user_id, 'user_name': user_name, 'user_screen_name':user_screen_name, 'user_location':user_location,
     'user_description':user_description, 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count, 
                  'user_created_at':user_created_at, 'user_protected':user_protected,
                  'user_verified': user_verified, 'user_geo_enabled':user_geo_enabled, 
                  'user_tweet_count':user_tweet_count, 'user_fav_count':user_fav_count}

    return dictionary

#### 1.2.1.2. Get followers/(followers+friends) ratio

In [28]:
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

#### 1.2.1.3. Get description length

In [29]:
def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len

#### 1.2.1.3. Get since

In [30]:
import time
from datetime import date
from datetime import datetime

In [31]:
def get_since(user_created_at):
    if user_created_at != None:
        creation_date = datetime.strptime(user_created_at ,'%a %b %d %H:%M:%S +0000 %Y')
        since = datetime.now() - creation_date
    else:
        since = 0
    return int(since.days)

#### 1.1.1.5. Get Number of Digits

In [32]:
def num_of_digits(username):
    count = 0

    for x in username:
        if x.isdigit():
            count+=1

    return count

### 1.1.2. Collect data using the functions above and transform into a Pandas DataFrame

In [33]:
dfPolitical = {'tweet_id':[],
              'is_retweet':[],
              'retweeted_username':[],
               'is_reply':[],
               'replied_username':[],
              'text':[],
              'num_mentions':[],
              'num_hashtags':[],
              'num_retweets':[],
              'num_favorites':[],
              'user_id':[],
              'user_screen_name':[],
              'user_description':[],
              'num_political_entities':[],
              'total_interactions':[],
              'punctuation': [],
              'tweet_length' : [],
              'is_verified': [],
                'is_political_mention': [],
                'description_entities':[],
               'description_length':[],
               'is_political_reply': [],
               'is_political_retweet':[],
               'avg_hash': [],
               
            'user_followers_count':[], 'user_friends_count':[], 
                  'user_created_at':[], 'user_protected':[],
                  'user_geo_enabled':[], 
                  'user_tweet_count':[], 'user_fav_count':[],'since':[]
               ,'average_tweet':[],'average_fav':[],'count_digits':[],'followers_to_all_ratio':[],
               'count_digits_tweets':[],'count_digits_desc':[],'punctuation_desc': [],'nonalpha': []
               
               
               
              }


with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
        
        
        dictionary = get_user_info_metadata_tweet(line)
        for k,v in dictionary.items():
            dfPolitical[k].append(v)
      
        
        # raw data:
        id_str = get_tweet_id(line)
        is_retweet, retweeted_username = check_if_retweet(line)
        is_reply, replied_username = check_if_reply(line)
        text = get_tweet_text(line)
        num_mentions, mention ,num_hashtags, avgh = get_number_mentions_hashtags(line)
        political_mention = check_political_mention(mention)
        political_reply = check_political_reply(replied_username)
        political_retweet = check_political_reply(retweeted_username)
        retweet_count, favorite_count = get_number_retweets_favorites(line)
        user_id_str, screen_name, user_description = get_user_info(line)
        punctuation = get_punc_num(text)
        is_verified = check_if_verified(line)
        tweet_length = len(text)
        description_length = len(user_description)
        # manually crafted data:
        num_political_entities = check_political_ent(text)
        total_num_interactions = total_interactions(retweet_count, favorite_count)
        dfPolitical['tweet_id'].append(id_str)
        dfPolitical['avg_hash'].append(avgh)
        dfPolitical['is_retweet'].append(is_retweet)
        dfPolitical['retweeted_username'].append(retweeted_username)
        dfPolitical['is_reply'].append(is_reply)
        dfPolitical['replied_username'].append(replied_username)
        dfPolitical['text'].append(text)
        dfPolitical['num_mentions'].append(num_mentions)
        dfPolitical['is_political_mention'].append(political_mention)
        dfPolitical['is_political_reply'].append(political_reply)
        dfPolitical['is_political_retweet'].append(political_retweet)
        dfPolitical['num_hashtags'].append(num_hashtags)
        dfPolitical['num_retweets'].append(retweet_count)
        dfPolitical['num_favorites'].append(favorite_count)
        dfPolitical['user_id'].append(user_id_str)
        dfPolitical['user_screen_name'].append(screen_name)
        dfPolitical['description_entities'].append(description_entities(user_description))
        dfPolitical['user_description'].append(user_description)
        dfPolitical['num_political_entities'].append(num_political_entities)
        dfPolitical['total_interactions'].append(total_num_interactions)
        dfPolitical['punctuation'].append(punctuation)
        dfPolitical['tweet_length'].append(tweet_length)
        dfPolitical['description_length'].append(tweet_length)
        dfPolitical['is_verified'].append(is_verified)
        dfPolitical['nonalpha'].append(nonalpha(text))
        dfPolitical['punctuation_desc'].append(get_punc_num(user_description))
        
          
        since = get_since(dictionary['user_created_at'])
        dfPolitical['since'].append(since)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], 
                                                      dictionary['user_friends_count'])
        
        dfPolitical['average_tweet'].append(dictionary['user_tweet_count'] / since)
        dfPolitical['average_fav'].append(dictionary['user_fav_count'] / since)
        
        dfPolitical['count_digits'].append(num_of_digits(screen_name))
        dfPolitical['count_digits_tweets'].append(num_of_digits(text))
        dfPolitical['count_digits_desc'].append(num_of_digits(user_description))

        dfPolitical['followers_to_all_ratio'].append(followers_all_ratio)

In [34]:
dfPolitical = pd.DataFrame(dfPolitical)
dfPolitical

,tweet_id,is_retweet,retweeted_username,is_reply,replied_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,punctuation,tweet_length,is_verified,is_political_mention,description_entities,description_length,is_political_reply,is_political_retweet,avg_hash,user_followers_count,user_friends_count,user_created_at,user_protected,user_geo_enabled,user_tweet_count,user_fav_count,since,average_tweet,average_fav,count_digits,followers_to_all_ratio,count_digits_tweets,count_digits_desc,punctuation_desc,nonalpha
0,1588568792984346624,0,,0,,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,1,147,3,140,False,0,1,140,0,0,0.0,284,539,Thu Oct 19 10:43:55 +0000 2017,False,True,2638,35673,1923,1.371815,18.550702,0,0.345079,1,0,0,21
1,1588452263047069697,0,,1,mahirunal,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,595514060,mtfdan,,1,0,2,97,False,0,0,97,0,0,0.0,131,589,Thu May 31 14:08:26 +0000 2012,False,False,6647,601,3890,1.708740,0.154499,0,0.181944,0,0,0,15
2,1569589330544398336,0,,0,,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,3,0,3,140,False,0,0,140,0,0,22.0,60,118,Mon Feb 01 22:56:06 +0000 2021,False,False,2924,1230,721,4.055479,1.705964,0,0.337079,2,0,0,26
3,1570428119609139201,0,,1,ajans_muhbir,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,1,0,2,140,False,0,0,140,0,0,0.0,2,69,Wed Jan 05 17:08:49 +0000 2022,False,False,1783,608,383,4.655352,1.587467,0,0.028169,1,0,0,26
4,1551163840368414722,0,,0,,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,1,0,3,140,False,0,0,140,0,0,0.0,119,166,Thu Apr 07 07:58:42 +0000 2022,False,False,7559,2427,292,25.886986,8.311644,8,0.417544,3,0,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33528,1568595408233832448,0,,0,,Gerçek kimlik taşımayan hesaplara cevap vermem...,0,0,9,81,576247173,ardanzenturk,RT ONAYLADIĞIM ANLAMINA GELMEZ\nArtık fikirler...,1,90,3,140,False,0,1,140,0,0,0.0,171399,4147,Thu May 10 11:47:23 +0000 2012,False,True,41342,4990,3911,10.570698,1.275889,0,0.976377,0,3,2,24
33529,1584027427696959488,0,,1,umitozdag,@umitozdag Neden Suriyelilerle ilgili bu kadar...,1,0,1,8,162308585,ozgul_61,Bridge design engineer Yaay hesabı : dilfiruz,3,9,2,140,False,2,0,140,1,0,0.0,3936,3227,Sat Jul 03 07:58:43 +0000 2010,False,True,52340,193138,4588,11.408021,42.096338,2,0.549490,4,0,1,29
33530,1585945783307730945,0,,1,celebimehmeta,@celebimehmeta Niye Türkiye yüzyılıda.Türkiye ...,1,0,0,1,415025519,ladrekova,,1,1,2,76,False,0,0,76,0,0,0.0,121,412,Thu Nov 17 20:15:55 +0000 2011,False,True,1225,1418,4085,0.299878,0.347124,0,0.227017,0,0,0,10
33531,1569748909521801221,1,muazzezeralp,0,,RT @muazzezeralp: @Doan58213655 @denizkonur @N...,7,1,6,0,1442125177727307781,yapikytgrivrlsn,,6,6,1,140,False,3,0,140,0,0,17.0,591,1003,Sun Sep 26 13:53:55 +0000 2021,False,False,153819,43990,485,317.152577,90.701031,0,0.370765,16,0,0,38


In [35]:
dfPolitical.describe()

,is_retweet,is_reply,num_mentions,num_hashtags,num_retweets,num_favorites,num_political_entities,total_interactions,punctuation,tweet_length,is_political_mention,description_entities,description_length,is_political_reply,is_political_retweet,avg_hash,user_followers_count,user_friends_count,user_tweet_count,user_fav_count,since,average_tweet,average_fav,count_digits,followers_to_all_ratio,count_digits_tweets,count_digits_desc,punctuation_desc,nonalpha
count,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.00000,33533.000000,3.353300e+04,33533.000000,3.353300e+04,3.353300e+04,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000,33533.000000
mean,0.137536,0.655742,1.446187,0.167358,173.729162,7.060388,2.505144,180.789551,1.573256,102.203919,1.075239,0.700385,102.203919,0.210062,0.04002,2.150042,1.343283e+04,1231.641905,3.181902e+04,2.753788e+04,1515.332180,34.351937,29.108197,1.960845,0.422808,2.261653,0.385710,0.435213,19.997823
std,0.344418,0.475133,1.567444,0.525753,1041.260855,108.140531,2.506287,1047.540331,1.065874,42.245978,1.556533,1.566080,42.245978,0.407358,0.19601,6.136218,1.780487e+05,3523.541544,5.684781e+05,6.434811e+04,1425.759163,193.211411,59.700815,2.772765,0.224282,2.961814,1.633528,0.770801,9.737021
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,4.000000,0.000000,0.00000,0.000000,0.000000e+00,0.000000,1.000000e+00,0.000000e+00,54.000000,0.000253,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,62.000000,0.000000,0.000000,62.000000,0.000000,0.00000,0.000000,8.500000e+01,143.000000,1.990000e+03,1.625000e+03,358.000000,1.921435,1.886516,0.000000,0.273381,0.000000,0.000000,0.000000,11.000000
50%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.000000,1.000000,2.000000,125.000000,0.000000,0.000000,125.000000,0.000000,0.00000,0.000000,2.660000e+02,369.000000,6.938000e+03,7.036000e+03,916.000000,7.970645,8.551132,1.000000,0.426174,1.000000,0.000000,0.000000,22.000000
75%,0.000000,1.000000,2.000000,0.000000,1.000000,1.000000,4.000000,5.000000,2.000000,140.000000,2.000000,1.000000,140.000000,0.000000,0.00000,0.000000,9.920000e+02,1041.000000,2.227700e+04,2.623000e+04,2525.000000,30.659220,29.851371,3.000000,0.516544,3.000000,0.000000,1.000000,27.000000
max,1.000000,1.000000,12.000000,12.000000,26401.000000,8469.000000,20.000000,26401.000000,5.000000,152.000000,15.000000,18.000000,152.000000,1.000000,1.00000,48.000000,9.213455e+06,181198.000000,5.169735e+07,3.141282e+06,5790.000000,16638.992276,3185.884381,14.000000,1.000000,55.000000,49.000000,5.000000,101.000000


## 1.2. From Users

### 1.2.1. Get user metadata from user_profiles.jsons.gz

In [36]:
dfBot = {'user_id':[],
         'user_name':[],
         'user_screen_name':[],
         'user_location':[],
         'user_description':[],
         'user_followers_count':[],
         'user_friends_count':[],
         'description_len':[],
         'followers_to_all_ratio':[],
        'user_created_at':[],
        'user_protected':[],
        'user_verified':[],
         'user_geo_enabled':[],
         'since':[],
         'user_tweet_count':[],
         'user_fav_count':[],
        'average_tweet': [],
        'average_fav': [],
        'count_digits': []}


with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)

        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        dfBot['description_len'].append(description_len)
        
        since = get_since(dictionary['user_created_at'])
        dfBot['since'].append(since)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], 
                                                      dictionary['user_friends_count'])
        
        dfBot['average_tweet'].append(dictionary['user_tweet_count'] / since)
        dfBot['average_fav'].append(dictionary['user_fav_count'] / since)
        
        dfBot['count_digits'].append(num_of_digits(dictionary['user_screen_name']))

        dfBot['followers_to_all_ratio'].append(followers_all_ratio)

In [37]:
dfBot = pd.DataFrame(dfBot)
dfBot

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,user_created_at,user_protected,user_verified,user_geo_enabled,since,user_tweet_count,user_fav_count,average_tweet,average_fav,count_digits
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,Fri Aug 27 13:07:30 +0000 2021,False,False,True,515,2551,17676,4.953398,34.322330,3
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,Fri Sep 11 08:45:44 +0000 2020,False,False,True,865,42771,15474,49.446243,17.889017,0
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,Wed Apr 10 18:15:31 +0000 2019,False,False,False,1384,14300,18220,10.332370,13.164740,8
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,Fri Jan 29 11:01:25 +0000 2016,False,False,False,2552,21303,26999,8.347571,10.579545,0
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,Sun Dec 01 18:16:41 +0000 2013,False,False,False,3340,1629,2179,0.487725,0.652395,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,0.513453,Mon Oct 26 21:08:22 +0000 2020,False,False,False,819,2396,10820,2.925519,13.211233,0
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,Wed Feb 03 18:39:01 +0000 2010,False,False,True,4737,75178,36671,15.870382,7.741398,0
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,0.451362,Fri May 01 13:56:23 +0000 2009,False,False,False,5016,6482,7389,1.292265,1.473086,0
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,Fri Mar 14 18:05:09 +0000 2014,False,False,True,3237,121113,140095,37.415199,43.279271,0


### 1.2.2. Get Tweet Info of Users in user_profiles.jsons.gz

#### 1.2.2.1. Check ratio of retweets to all tweets

In [38]:
def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio

#### 1.2.2.2. Check median number of favorites

In [39]:
def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

In [40]:
def get_avg_length(line):
    if len(line["tweets"]) == 0:
        return 0

    avg = np.mean([len(tweet['text']) for tweet in line['tweets']])

    return avg

#check max length of tweets

def get_max_length(line):

    if len(line["tweets"]) == 0:
        return 0

    m = np.max([len(tweet['text']) for tweet in line['tweets']])

    return m


def get_std_length(line):

    if len(line["tweets"]) == 0:
        return 0

    m = np.std([len(tweet['text']) for tweet in line['tweets']])

    return m

In [41]:
import re, math 
from collections import Counter 
WORD = re.compile(r'\w+') 
def cosine_sim(v1, v2): 
     intersection = set(v1.keys()) & set(v2.keys()) 
     nume = sum([v1[x] * v2[x] for x in intersection]) 
     test1 = sum([v1[x]**2 for x in v1.keys()]) 
     test2 = sum([v2[x]**2 for x in v2.keys()]) 
     den = math.sqrt(test1) * math.sqrt(test2) 
     if not den: 
        return 0.0 
     else: 
        return float(nume) / den 
def vectorize_text(text): 
     words = WORD.findall(text) 
     return Counter(words) 

def avg_cosine_sim(line):
    tweets = line['tweets']
    sims = []
    for x in range(len(tweets) - 3):
        tweet = tweets[x]['text']
        tweet2 = tweets[x + 1]['text']
        tweet3 = tweets[x + 2]['text']
        tweet4 = tweets[x + 3]['text']

        v1 = vectorize_text(tweet)
        v2 = vectorize_text(tweet2)
        v3 = vectorize_text(tweet3)
        v4 = vectorize_text(tweet4)

        cosim = cosine_sim(v1, v2)
        cosim2 = cosine_sim(v1, v3)
        cosim3 = cosine_sim(v1, v4)
        
        cosim = max([cosim, cosim2, cosim3])

        sims.append(cosim)

    avg = np.mean(sims)

    return avg


def countlinks(line):
    count = 0
    try:
        for x in line['tweets']:

            try:
                tweet = x['entities']['urls']
                count += len(tweet)
            except:
                continue
        
    except:
        return count

    return count




def tweetvelocity(line):

    if len(line['tweets']) == 0:
        return 1

    count = 0
    arr = []
    for x in range(len(line['tweets']) - 1):

        t = pd.to_datetime(line['tweets'][x]["created_at"])
        t2 = pd.to_datetime(line['tweets'][x+1]["created_at"])

        v = t - t2

        arr.append(v.days)

    return np.mean(arr)
        

def tweetvelocity_2(line):

    if len(line['tweets']) == 0:
        return 200

    t = pd.to_datetime(line['tweets'][0]["created_at"])
    t2 = pd.to_datetime(line['tweets'][-1]["created_at"])  

    return (t - t2).days


def media_count(line):
    count = 0
    for tweet in line["tweets"]:
        try: 
            y = len(tweet["entities"]["media"])
            count += 1
        except:
            continue

    return count

def count_quote(line):
    count = 0
    for tweet in line['tweets']:
        if tweet["is_quote_status"]:
            count += 1

    return count

### 1.2.3. Collect data using the functions above and transform into a Pandas DataFrame

In [42]:
dfBotTweets = {'user_id':[],
               'retweet_total_ratio':[],
               'num_median_favorites':[],
               'num_of_tweets':[],
               'avg_tweet_length':[],
               'max_tweet_length':[],
               'std_tweet_length':[],
               'avg_cosine_sim': [],
               'link_count': [],
               'media': [],
               'quote_tweets': []
              }

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)
        
        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)
        
        dfBotTweets['num_of_tweets'].append(len(line['tweets']))
        
        avg_tweet_length = get_avg_length(line)
        dfBotTweets['avg_tweet_length'].append(avg_tweet_length)

        max_tweet_length = get_max_length(line)
        dfBotTweets['max_tweet_length'].append(max_tweet_length)

        std_tweet_length = get_std_length(line)
        dfBotTweets['std_tweet_length'].append(std_tweet_length)
        
        avg_cosine = avg_cosine_sim(line)
        dfBotTweets['avg_cosine_sim'].append(avg_cosine)

        link_count = countlinks(line)
        dfBotTweets['link_count'].append(link_count)

        media = media_count(line)
        dfBotTweets['media'].append(media)

        quote_tweets = count_quote(line)
        dfBotTweets['quote_tweets'].append(quote_tweets)

        i += 1
        if i % 1000 == 0:
            print(i)

C:\Users\Gamegaraj\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Gamegaraj\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [43]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

,user_id,retweet_total_ratio,num_median_favorites,num_of_tweets,avg_tweet_length,max_tweet_length,std_tweet_length,avg_cosine_sim,link_count,media,quote_tweets
0,594642154,0.115000,2.0,200,87.340000,140,46.855249,0.121920,70,5,36
1,525600289,0.005025,1.0,199,49.351759,140,38.673961,0.140806,21,10,0
2,931895965501534209,0.900000,0.0,200,132.025000,140,22.288885,0.296732,10,9,3
3,1591543462746329088,0.185000,0.0,200,119.200000,140,30.044301,0.197845,84,15,12
4,734801354749796352,1.000000,0.0,200,122.155000,140,28.996223,0.141741,8,52,14
...,...,...,...,...,...,...,...,...,...,...,...
28310,1591370361488252928,0.800000,0.0,200,128.135000,140,18.832599,0.287320,24,0,2
28311,1475272459616235525,0.825000,0.0,200,119.580000,144,34.269864,0.179948,10,14,10
28312,1096753792731750401,0.051020,1.0,196,69.673469,140,39.701636,0.124059,42,10,30
28313,1269527617687953409,0.095000,2.0,200,51.130000,140,31.249370,0.051498,7,15,6


In [44]:
dfBotTweets.isna().sum()

user_id                   0
retweet_total_ratio      83
num_median_favorites     83
num_of_tweets             0
avg_tweet_length          0
max_tweet_length          0
std_tweet_length          0
avg_cosine_sim          190
link_count                0
media                     0
quote_tweets              0
dtype: int64

### 1.2.3. Merge dfBot and dfBotTweets

In [45]:
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')

#dfBotAll[['retweet_total_ratio', 'num_median_favorites']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites']].fillna(0)
dfBotAll['num_median_favorites'] = dfBotAll['num_median_favorites'].fillna((dfBotAll['num_median_favorites'].mean()))
dfBotAll['retweet_total_ratio'] = dfBotAll['retweet_total_ratio'].fillna((dfBotAll['retweet_total_ratio'].mean()))
dfBotAll[['user_protected', 'user_verified','user_geo_enabled']] = dfBotAll[['user_protected', 'user_verified','user_geo_enabled']].astype(int)
dfBotAll

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,user_created_at,user_protected,user_verified,user_geo_enabled,since,user_tweet_count,user_fav_count,average_tweet,average_fav,count_digits,retweet_total_ratio,num_median_favorites,num_of_tweets,avg_tweet_length,max_tweet_length,std_tweet_length,avg_cosine_sim,link_count,media,quote_tweets
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,Fri Aug 27 13:07:30 +0000 2021,0,0,1,515,2551,17676,4.953398,34.322330,3,0.395939,0.0,197.0,72.126904,140.0,38.729298,0.120227,1.0,47.0,2.0
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,Fri Sep 11 08:45:44 +0000 2020,0,0,1,865,42771,15474,49.446243,17.889017,0,0.125000,0.0,200.0,39.605000,140.0,33.943025,0.089971,20.0,20.0,11.0
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,Wed Apr 10 18:15:31 +0000 2019,0,0,0,1384,14300,18220,10.332370,13.164740,8,0.910000,0.0,200.0,121.570000,140.0,32.988712,0.251797,29.0,7.0,54.0
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,Fri Jan 29 11:01:25 +0000 2016,0,0,0,2552,21303,26999,8.347571,10.579545,0,0.015306,1.0,196.0,83.020408,140.0,39.681379,0.191220,114.0,18.0,89.0
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,Sun Dec 01 18:16:41 +0000 2013,0,0,0,3340,1629,2179,0.487725,0.652395,0,0.659898,0.0,197.0,119.421320,140.0,33.019455,0.161071,45.0,14.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,0.513453,Mon Oct 26 21:08:22 +0000 2020,0,0,0,819,2396,10820,2.925519,13.211233,0,0.015000,1.0,200.0,70.435000,140.0,38.439378,0.159389,30.0,13.0,3.0
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,Wed Feb 03 18:39:01 +0000 2010,0,0,1,4737,75178,36671,15.870382,7.741398,0,0.291457,2.0,199.0,88.246231,140.0,44.701856,0.187754,79.0,13.0,64.0
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,0.451362,Fri May 01 13:56:23 +0000 2009,0,0,0,5016,6482,7389,1.292265,1.473086,0,0.061538,0.0,195.0,42.805128,140.0,26.339265,0.174823,3.0,6.0,0.0
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,Fri Mar 14 18:05:09 +0000 2014,0,0,1,3237,121113,140095,37.415199,43.279271,0,0.995000,0.0,200.0,129.175000,140.0,23.942731,0.185794,7.0,24.0,7.0


#### DF Preprocessings

In [46]:
dfPolitical[['user_protected', 'is_verified','user_geo_enabled']] = dfPolitical[['user_protected', 'is_verified','user_geo_enabled']].astype(int)

In [47]:
dfPolitical['is_verified'] = dfPolitical['is_verified'].astype(int)
dfPolitical['is_political_mention'] = dfPolitical['is_political_mention'].astype(int)
dfPolitical['is_political_reply'] = dfPolitical['is_political_reply'].astype(int)

In [48]:
dfBotAll['num_of_tweets'] = dfBotAll['num_of_tweets'].fillna((dfBotAll['num_of_tweets'].mean()))
dfBotAll['avg_tweet_length'] = dfBotAll['avg_tweet_length'].fillna((dfBotAll['avg_tweet_length'].mean()))
dfBotAll['max_tweet_length'] = dfBotAll['max_tweet_length'].fillna((dfBotAll['max_tweet_length'].mean()))
dfBotAll['std_tweet_length'] = dfBotAll['std_tweet_length'].fillna((dfBotAll['std_tweet_length'].mean()))
dfBotAll['avg_cosine_sim'] = dfBotAll['avg_cosine_sim'].fillna((dfBotAll['avg_cosine_sim'].mean()))
dfBotAll['link_count'] = dfBotAll['link_count'].fillna((dfBotAll['link_count'].mean()))
dfBotAll['media'] = dfBotAll['media'].fillna((dfBotAll['media'].mean()))
dfBotAll['quote_tweets'] = dfBotAll['quote_tweets'].fillna((dfBotAll['quote_tweets'].mean()))

# 2. TRAIN MODEL

## 2.1. Political Tweet Prediction

### 2.1.1. Merge dfPolitical data with labels

In [49]:
#dfPoliticalAll_train = dfPolitical.merge(dfBotTweets,on='user_id')

In [50]:
dfPoliticalAll_train = dfPolitical.merge(trainingTweetDf,
                                         on='tweet_id')

dfPoliticalAll_train.head()

,tweet_id,is_retweet,retweeted_username,is_reply,replied_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,punctuation,tweet_length,is_verified,is_political_mention,description_entities,description_length,is_political_reply,is_political_retweet,avg_hash,user_followers_count,user_friends_count,user_created_at,user_protected,user_geo_enabled,user_tweet_count,user_fav_count,since,average_tweet,average_fav,count_digits,followers_to_all_ratio,count_digits_tweets,count_digits_desc,punctuation_desc,nonalpha,isPolitical
0,1588568792984346624,0,,0,,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,1,147,3,140,0,0,1,140,0,0,0.0,284,539,Thu Oct 19 10:43:55 +0000 2017,0,1,2638,35673,1923,1.371815,18.550702,0,0.345079,1,0,0,21,No
1,1588568792984346624,0,,0,,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,1,147,3,140,0,0,1,140,0,0,0.0,284,539,Thu Oct 19 10:43:55 +0000 2017,0,1,2638,35673,1923,1.371815,18.550702,0,0.345079,1,0,0,21,Yes
2,1588568792984346624,0,,0,,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,1,147,3,140,0,0,1,140,0,0,0.0,284,539,Thu Oct 19 10:43:55 +0000 2017,0,1,2638,35673,1923,1.371815,18.550702,0,0.345079,1,0,0,21,Yes
3,1588568792984346624,0,,0,,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,1,147,3,140,0,0,1,140,0,0,0.0,284,539,Thu Oct 19 10:43:55 +0000 2017,0,1,2638,35673,1923,1.371815,18.550702,0,0.345079,1,0,0,21,No
4,1588568792984346624,0,,0,,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,1,147,3,140,0,0,1,140,0,0,0.0,284,539,Thu Oct 19 10:43:55 +0000 2017,0,1,2638,35673,1923,1.371815,18.550702,0,0.345079,1,0,0,21,Yes


In [51]:
dfPoliticalAll_train.describe()

,is_retweet,is_reply,num_mentions,num_hashtags,num_retweets,num_favorites,num_political_entities,total_interactions,punctuation,tweet_length,is_verified,is_political_mention,description_entities,description_length,is_political_reply,is_political_retweet,avg_hash,user_followers_count,user_friends_count,user_protected,user_geo_enabled,user_tweet_count,user_fav_count,since,average_tweet,average_fav,count_digits,followers_to_all_ratio,count_digits_tweets,count_digits_desc,punctuation_desc,nonalpha
count,4394.0,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.0,4394.000000,4.394000e+03,4394.000000,4394.0,4394.000000,4.394000e+03,4.394000e+03,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.00000,4394.000000,4394.000000
mean,0.0,0.755121,1.290396,0.145881,1.419436,7.051661,2.188666,8.471097,1.505462,98.163177,0.017979,0.922394,0.686846,98.163177,0.248521,0.0,1.883741,1.314634e+04,1102.630178,0.0,0.293127,2.385732e+04,2.471766e+04,1532.160218,29.271619,26.107462,1.916022,0.416134,2.171825,0.39076,0.441284,19.145426
std,0.0,0.430065,1.430653,0.497087,12.319371,96.480073,2.336996,107.509267,1.086134,43.006445,0.132890,1.446090,1.616209,43.006445,0.432204,0.0,5.867622,1.510670e+05,3095.962230,0.0,0.455248,6.594234e+04,5.712725e+04,1440.127179,62.012657,55.058641,2.772750,0.226633,2.717827,1.61247,0.777757,9.674913
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.0,0.000000,1.000000e+00,0.000000e+00,55.000000,0.001313,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,56.000000,0.000000,0.000000,0.000000,56.000000,0.000000,0.0,0.000000,8.300000e+01,140.000000,0.0,0.000000,1.964000e+03,1.316500e+03,356.000000,1.811811,1.615565,0.000000,0.260919,0.000000,0.00000,0.000000,11.000000
50%,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.000000,113.000000,0.000000,0.000000,0.000000,113.000000,0.000000,0.0,0.000000,2.480000e+02,356.000000,0.0,0.000000,6.333000e+03,6.187000e+03,927.000000,6.989890,7.325701,0.000000,0.417014,1.000000,0.00000,0.000000,20.000000
75%,0.0,1.000000,1.000000,0.000000,0.000000,1.000000,3.000000,2.000000,2.000000,140.000000,0.000000,2.000000,1.000000,140.000000,0.000000,0.0,0.000000,8.765000e+02,944.750000,0.0,1.000000,1.904750e+04,2.199000e+04,2538.250000,27.471180,24.915562,2.000000,0.516582,3.000000,0.00000,1.000000,26.000000
max,0.0,1.000000,10.000000,7.000000,600.000000,5722.000000,14.000000,6322.000000,5.000000,152.000000,1.000000,11.000000,14.000000,152.000000,1.000000,0.0,44.000000,3.509923e+06,84879.000000,0.0,1.000000,1.090423e+06,1.020739e+06,5790.000000,1210.622699,899.307692,13.000000,1.000000,28.000000,35.00000,4.000000,71.000000


In [52]:
dfPoliticalAll_train.shape

(4394, 41)

### 2.1.2. Separate X and y values
We only use 3 features here to create a baseline model. However, it is not enough to get good results.

In [53]:
#X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags']]
X = dfPoliticalAll_train.select_dtypes(exclude=['object'])
#X = dfPoliticalAll_train[['is_retweet','is_reply','num_mentions','num_hashtags','num_retweets','num_favorites','num_political_entities','total_interactions','punctuation','tweet_length','is_political_mention','is_political_reply','user_followers_count','user_friends_count','user_tweet_count','user_fav_count','since','average_tweet','average_fav','count_digits_tweets',  ]]
y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)

In [54]:
X_binary = X[['is_verified','is_political_reply','is_political_retweet','user_protected','user_geo_enabled']]
X_numerical = X.drop(['is_verified','is_political_reply','is_political_retweet','user_protected','user_geo_enabled'], axis=1)

In [55]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_numerical = ss.fit_transform(X_numerical)
X = np.concatenate((X_binary, X_numerical), axis=1)

In [56]:
X.shape

(4394, 32)

#### CFS

In [57]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# create the feature selector
selector = SelectKBest(score_func=mutual_info_classif, k=10)

# fit the feature selector on the data
selector.fit(X, y)

# get the selected features
X = selector.transform(X)

### 2.1.3. Train - validation split

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_valid2, y_train, y_valid2 = train_test_split(X, y, test_size=0.20, random_state=42)

In [59]:
X_train.shape

(3515, 10)

### 2.1.4. Train the model

Here, you may use different models such as neural networks, XGBoost, AdaBoost, RandomForest, Linear Regression, Logistic Regression etc. to see which model does the best. Also, you can use grid_search_cv() or a basic for loop to optimize the hyperparameters of your model.

In [60]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

# create the model
model1 = Sequential()
model1.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(16, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))

# compile the model
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model1.fit(X_train, y_train, epochs=69, batch_size=64)
preds = model1.predict(X_valid2)

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Epoch 1/69
55/55 [==============================] - 1s 1ms/step - loss: 0.6073 - accuracy: 0.6865
Epoch 2/69
55/55 [==============================] - 0s 1ms/step - loss: 0.5686 - accuracy: 0.7289
Epoch 3/69
55/55 [==============================] - 0s 1ms/step - loss: 0.5473 - accuracy: 0.7462
Epoch 4/69
55/55 [==============================] - 0s 1ms/step - loss: 0.5382 - accuracy: 0.7471
Epoch 5/69
55/55 [==============================] - 0s 1ms/step - loss: 0.5348 - accuracy: 0.7548
Epoch 6/69
55/55 [==============================] - 0s 1ms/step - loss: 0.5251 - accuracy: 0.7550
Epoch 7/69
55/55 [==============================] - 0s 1ms/step - loss: 0.5294 - accuracy: 0.7522
Epoch 8/69
55/55 [==============================] - 0s 1ms/step - loss: 0.5205 - accuracy: 0.7505
Epoch 9/69
55/55 [==============================] - 0s 1ms/step - loss: 0.5181 - accuracy: 0.7514
Epoch 10/69
55/55 [==============================] - 0s 1ms/step - loss: 0.5170 - accuracy: 0.7576
Epoch 11/69
55/55 [

In [61]:
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

In [62]:
# from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
mse = mean_squared_error(y_valid2, preds)
mse

0.15334016642355136

In [63]:
max(preds)

array([0.977548], dtype=float32)

In [64]:
min(preds)

array([0.00311425], dtype=float32)

In [65]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

# Initialize the models
ada = AdaBoostClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()
rf = RandomForestClassifier()

# Create the ensemble model
ensemble = VotingClassifier(estimators=[('ada', ada), ('xgb', xgb), ('lgb', lgb), ('rf', rf)], voting='soft')

# Fit the ensemble model on the training data
ensemble.fit(X_train, y_train)

# Make predictions on the test data
y_pred = ensemble.predict(X_valid2)

# Print the accuracy
print("Accuracy:", ensemble.score(X_valid2, y_valid2))

Accuracy: 0.7747440273037542


In [66]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

# Initialize the models
ada = AdaBoostClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()
rf = RandomForestClassifier()

# Create the ensemble model
ensemble = VotingClassifier(estimators=[('ada', ada), ('xgb', xgb), ('lgb', lgb), ('rf', rf)], voting='soft')

# Define the base estimator (e.g. decision tree)
base_estimator = ensemble

# Define the bagging classifier
bagging_clf = BaggingClassifier(base_estimator=base_estimator, n_estimators=50, max_samples=0.8, max_features=0.8)

# Fit the bagging classifier to the training data
bagging_clf.fit(X_train, y_train)

# Predict on the test data
y_pred = bagging_clf.predict(X_valid2)
print("Accuracy:", bagging_clf.score(X_valid2, y_valid2))

Accuracy: 0.7952218430034129


## 2.2. Bot Detection

### 2.2.1. Merge dfBotAll data with labels

In [67]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()

In [68]:
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')

dfBotAll_train

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,user_created_at,user_protected,user_verified,user_geo_enabled,since,user_tweet_count,user_fav_count,average_tweet,average_fav,count_digits,retweet_total_ratio,num_median_favorites,num_of_tweets,avg_tweet_length,max_tweet_length,std_tweet_length,avg_cosine_sim,link_count,media,quote_tweets,screen_name,isBot
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,Fri Aug 27 13:07:30 +0000 2021,0,0,1,515,2551,17676,4.953398,34.322330,3,0.395939,0.0,197.0,72.126904,140.0,38.729298,0.120227,1.0,47.0,2.0,nasreenakhan006,No
1,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,Fri Aug 27 13:07:30 +0000 2021,0,0,1,515,2551,17676,4.953398,34.322330,3,0.395939,0.0,197.0,72.126904,140.0,38.729298,0.120227,1.0,47.0,2.0,nasreenakhan006,No
2,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,Fri Aug 27 13:07:30 +0000 2021,0,0,1,515,2551,17676,4.953398,34.322330,3,0.395939,0.0,197.0,72.126904,140.0,38.729298,0.120227,1.0,47.0,2.0,nasreenakhan006,No
3,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,Fri Sep 11 08:45:44 +0000 2020,0,0,1,865,42771,15474,49.446243,17.889017,0,0.125000,0.0,200.0,39.605000,140.0,33.943025,0.089971,20.0,20.0,11.0,scorpiehoez,No
4,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,Fri Sep 11 08:45:44 +0000 2020,0,0,1,865,42771,15474,49.446243,17.889017,0,0.125000,0.0,200.0,39.605000,140.0,33.943025,0.089971,20.0,20.0,11.0,scorpiehoez,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4281,1286770207134973954,Hamide Arabacı,anka6054,,,151,61,0,0.712264,Fri Jul 24 21:08:34 +0000 2020,0,0,0,913,5288,5347,5.791895,5.856517,4,0.000000,1.0,200.0,120.190000,140.0,23.062825,0.878755,124.0,0.0,0.0,anka6054,No
4282,1598032338323214338,atamabekleyenbahceci,atamabekleyenzz,,,173,367,0,0.320370,Wed Nov 30 19:13:03 +0000 2022,0,0,0,54,311,196,5.759259,3.629630,0,0.580000,0.0,200.0,131.670000,140.0,18.445897,0.635663,84.0,0.0,4.0,atamabekleyenzz,No
4283,760235343966863360,Emrah İNCİ,memrahinci,Istanbul - Bayburt,Researcher | Middle East | Political Science |...,5863,5905,71,0.498215,Mon Aug 01 22:06:45 +0000 2016,0,0,1,2366,1029,725,0.434911,0.306424,0,0.040000,36.0,200.0,127.080000,144.0,30.592378,0.242883,161.0,30.0,3.0,memrahinci,No
4284,1553973684100124672,Murat Kkk,muratkkk18,,Normal sıradan bir insanım,1,10,26,0.090909,Mon Aug 01 05:19:56 +0000 2022,0,0,0,176,18,38,0.102273,0.215909,2,0.769231,0.0,13.0,115.307692,140.0,31.972066,0.171521,2.0,2.0,0.0,muratkkk18,No


In [69]:
trainingUserDf.isBot.value_counts()

No     3406
Yes     880
Name: isBot, dtype: int64

### 2.2.2. Separate X and y values
We use only 4 features here to create a baseline model. However, it is not enough to get good results.

In [70]:
#X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites']]
X = dfBotAll_train.select_dtypes(exclude=['object'])
y = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

In [71]:
X_binary = X[['user_verified','user_protected','user_geo_enabled']]
X_numerical = X.drop(['user_verified','user_protected','user_geo_enabled'], axis=1)

In [72]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_numerical = ss.fit_transform(X_numerical)
X = np.concatenate((X_binary, X_numerical), axis=1)

In [73]:
X.shape

(4286, 23)

#### CFS

In [74]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# create the feature selector
selector = SelectKBest(score_func=mutual_info_classif, k=18)
# fit the feature selector on the data
selector.fit(X, y)

# get the selected features
X = selector.transform(X)

### 2.2.3. Train-test split

In [75]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

In [76]:
X_train.shape

(3428, 18)

### 2.2.4. Train the model

In [77]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

# create the model
model2 = Sequential()
model2.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(16, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid'))

# compile the model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model2.fit(X_train, y_train, epochs=69, batch_size=64)
preds = model2.predict(X_valid)

Epoch 1/69
54/54 [==============================] - 0s 1ms/step - loss: 0.5769 - accuracy: 0.7471
Epoch 2/69
54/54 [==============================] - 0s 1ms/step - loss: 0.4421 - accuracy: 0.7993
Epoch 3/69
54/54 [==============================] - 0s 1ms/step - loss: 0.4151 - accuracy: 0.8051
Epoch 4/69
54/54 [==============================] - 0s 1ms/step - loss: 0.3988 - accuracy: 0.8086
Epoch 5/69
54/54 [==============================] - 0s 1ms/step - loss: 0.3948 - accuracy: 0.8232
Epoch 6/69
54/54 [==============================] - 0s 1ms/step - loss: 0.3914 - accuracy: 0.8361
Epoch 7/69
54/54 [==============================] - 0s 2ms/step - loss: 0.3827 - accuracy: 0.8343
Epoch 8/69
54/54 [==============================] - 0s 2ms/step - loss: 0.3803 - accuracy: 0.8372
Epoch 9/69
54/54 [==============================] - 0s 1ms/step - loss: 0.3791 - accuracy: 0.8358
Epoch 10/69
54/54 [==============================] - 0s 1ms/step - loss: 0.3759 - accuracy: 0.8352
Epoch 11/69
54/54 [

In [78]:
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

In [79]:
# from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
mse = mean_squared_error(y_valid, preds)
mse

0.13183163005375742

In [80]:
max(preds)

array([0.86660194], dtype=float32)

In [81]:
min(preds)

array([1.5077019e-09], dtype=float32)

In [82]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

# Initialize the models
ada = AdaBoostClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()
rf = RandomForestClassifier()

# Create the ensemble model
ensemble = VotingClassifier(estimators=[('ada', ada), ('xgb', xgb), ('lgb', lgb), ('rf', rf)], voting='soft')

# Fit the ensemble model on the training data
ensemble.fit(X_train, y_train)

# Make predictions on the test data
y_pred = ensemble.predict(X_valid)

# Print the accuracy
print("Accuracy:", ensemble.score(X_valid, y_valid))

Accuracy: 0.7983682983682984


In [83]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization
from keras.callbacks import EarlyStopping
# Define the model architecture
model = Sequential()

# Add convolutional layers
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(18, 1)))
model.add(BatchNormalization())
model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv1D(filters=256, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(filters=256, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))

# Add dense layers
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Add output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0.2, callbacks=[early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_valid)

Epoch 1/50
86/86 [==============================] - 3s 10ms/step - loss: 0.6536 - accuracy: 0.7082 - val_loss: 0.4604 - val_accuracy: 0.8265
Epoch 2/50
86/86 [==============================] - 1s 8ms/step - loss: 0.5170 - accuracy: 0.7863 - val_loss: 0.4342 - val_accuracy: 0.8265
Epoch 3/50
86/86 [==============================] - 1s 8ms/step - loss: 0.4709 - accuracy: 0.7921 - val_loss: 0.4052 - val_accuracy: 0.8280
Epoch 4/50
86/86 [==============================] - 1s 8ms/step - loss: 0.4387 - accuracy: 0.8129 - val_loss: 0.3969 - val_accuracy: 0.8440
Epoch 5/50
86/86 [==============================] - 1s 8ms/step - loss: 0.4284 - accuracy: 0.8063 - val_loss: 0.4047 - val_accuracy: 0.8397
Epoch 6/50
86/86 [==============================] - 1s 8ms/step - loss: 0.4142 - accuracy: 0.8195 - val_loss: 0.3524 - val_accuracy: 0.8528
Epoch 7/50
86/86 [==============================] - 1s 8ms/step - loss: 0.4122 - accuracy: 0.8111 - val_loss: 0.3458 - val_accuracy: 0.8499
Epoch 8/50
86/86 [=

In [84]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

# Initialize the models
ada = AdaBoostClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()
rf = RandomForestClassifier()

# Create the ensemble model
ensemble = VotingClassifier(estimators=[('ada', ada), ('xgb', xgb), ('lgb', lgb), ('rf', rf)], voting='soft')

# Define the base estimator (e.g. decision tree)
base_estimator = ensemble

# Define the bagging classifier
bagging_clf = BaggingClassifier(base_estimator=base_estimator, n_estimators=100, max_samples=0.8, max_features=0.8)

# Fit the bagging classifier to the training data
bagging_clf.fit(X_train, y_train)

# Predict on the test data
y_pred = bagging_clf.predict(X_valid)
print("Accuracy:", bagging_clf.score(X_valid, y_valid))

Accuracy: 0.8018648018648019


# 3. MAKE PREDICTIONS

Here, you will make predictions with the models that you have trained above.

## 3.1. Predictions for Tweets (Political or Not)

In [85]:
# read the evaluation file as follows
evaluationTweetDf = pd.read_csv(DATA_PATH+'evaluation-round{}-tweet.csv'.format(ROUND), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()
evaluationTweetDf

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X_pol = dfPolitical_test.select_dtypes(exclude=['object'])



In [86]:
#X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags']]
Xtr_pol = dfPoliticalAll_train.select_dtypes(exclude=['object'])
ytr_pol = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)

In [87]:
X_binary = Xtr_pol[['is_verified','is_political_reply','is_political_retweet','user_protected','user_geo_enabled']]
X_numerical = Xtr_pol.drop(['is_verified','is_political_reply','is_political_retweet','user_protected','user_geo_enabled'], axis=1)

X_binary_t = X_pol[['is_verified','is_political_reply','is_political_retweet','user_protected','user_geo_enabled']]
X_numerical_t = X_pol.drop(['is_verified','is_political_reply','is_political_retweet','user_protected','user_geo_enabled'], axis=1)

In [88]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_numerical = ss.fit_transform(X_numerical)
X_numerical_t = ss.fit_transform(X_numerical_t)

Xtr_pol = np.concatenate((X_binary, X_numerical), axis=1)
X_pol = np.concatenate((X_binary_t, X_numerical_t), axis=1)

In [89]:
Xtr_pol.shape

(4394, 32)

In [90]:
X_pol.shape

(5000, 32)

#### CFS

In [91]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# create the feature selector
selector = SelectKBest(score_func=mutual_info_classif, k=10)

# fit the feature selector on the data
selector.fit(Xtr_pol, ytr_pol)

# get the selected features
Xtr_pol = selector.transform(Xtr_pol)
X_pol = selector.transform(X_pol)

In [92]:
Xtr_pol.shape

(4394, 10)

In [93]:
X_train, X_valid5, y_train, y_valid5 = train_test_split(Xtr_pol, ytr_pol, test_size=0.20, random_state=42)

In [94]:
X_pol.shape

(5000, 10)

In [95]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

# create the model
model1 = Sequential()
model1.add(Dense(64, input_dim=10, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(16, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))

# compile the model
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model1.fit(Xtr_pol,ytr_pol, epochs=69, batch_size=64)

Epoch 1/69
69/69 [==============================] - 1s 1ms/step - loss: 0.6110 - accuracy: 0.6570
Epoch 2/69
69/69 [==============================] - 0s 1ms/step - loss: 0.5735 - accuracy: 0.7148
Epoch 3/69
69/69 [==============================] - 0s 1ms/step - loss: 0.5517 - accuracy: 0.7387
Epoch 4/69
69/69 [==============================] - 0s 1ms/step - loss: 0.5406 - accuracy: 0.7451
Epoch 5/69
69/69 [==============================] - 0s 2ms/step - loss: 0.5413 - accuracy: 0.7497
Epoch 6/69
69/69 [==============================] - 0s 2ms/step - loss: 0.5367 - accuracy: 0.7565
Epoch 7/69
69/69 [==============================] - 0s 2ms/step - loss: 0.5244 - accuracy: 0.7569
Epoch 8/69
69/69 [==============================] - 0s 2ms/step - loss: 0.5190 - accuracy: 0.7590
Epoch 9/69
69/69 [==============================] - 0s 2ms/step - loss: 0.5251 - accuracy: 0.7590
Epoch 10/69
69/69 [==============================] - 0s 2ms/step - loss: 0.5291 - accuracy: 0.7615
Epoch 11/69
69/69 [

In [96]:
# make predictions based on these variables
predictions_political = model1.predict(X_pol)

157/157 [==============================] - 0s 629us/step


In [97]:
max(predictions_political)

array([0.9998972], dtype=float32)

In [98]:
predictions_political

array([[0.2681775 ],
       [0.35402843],
       [0.91055363],
       ...,
       [0.36445564],
       [0.08808623],
       [0.868545  ]], dtype=float32)

In [99]:
min(predictions_political)

array([0.0305412], dtype=float32)

In [100]:
max(predictions_political)

array([0.9998972], dtype=float32)

### This part is important! We expect you to return your predictions in the following format:

In [101]:
modelPredTweet = dict([(x,float(y)) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])
modelPredTweet

{'1434787703783051264': 0.26817750930786133,
 '1367571642604544000': 0.3540284335613251,
 '1589993032975544320': 0.9105536341667175,
 '1565312596135354373': 0.822589099407196,
 '1579558096833511424': 0.7545415759086609,
 '1439547067337256967': 0.24057456851005554,
 '1559963768372740098': 0.7425578236579895,
 '1562853131251118081': 0.7011553645133972,
 '1586021183958704128': 0.7671231627464294,
 '1585766233491886081': 0.8263567090034485,
 '1427746815420604417': 0.40334293246269226,
 '1352635736537882629': 0.4674953818321228,
 '1415032260571680768': 0.3925659656524658,
 '1548636597628899328': 0.7297621369361877,
 '1564926450096013313': 0.35736000537872314,
 '1585634359612420101': 0.838631272315979,
 '1597138789108895744': 0.831354022026062,
 '1391681495622995971': 0.18465213477611542,
 '1389951943343316995': 0.13016429543495178,
 '1452348722810138646': 0.559519350528717,
 '1595829502021623812': 0.5541213750839233,
 '1413108476348354562': 0.13664542138576508,
 '1579408398894137344': 0.830

## 3.2. Predictions for Users (Bot or Not)

In [102]:
evaluationUserDf = pd.read_csv(DATA_PATH+'evaluation-round{}-user.csv'.format(ROUND), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X_bot = dfBot_test.select_dtypes(exclude=['object'])



In [103]:
#X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites']]
Xtr_bot = dfBotAll_train.select_dtypes(exclude=['object'])
ytr_bot = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

In [104]:
X_binary = Xtr_bot[['user_verified','user_protected','user_geo_enabled']]
X_numerical = Xtr_bot.drop(['user_verified','user_protected','user_geo_enabled'], axis=1)

X_binary_t = X_bot[['user_verified','user_protected','user_geo_enabled']]
X_numerical_t = X_bot.drop(['user_verified','user_protected','user_geo_enabled'], axis=1)



In [105]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_numerical = ss.fit_transform(X_numerical)
Xtr_bot = np.concatenate((X_binary, X_numerical), axis=1)

X_numerical_t = ss.fit_transform(X_numerical_t)
X_bot = np.concatenate((X_binary_t, X_numerical_t), axis=1)

In [106]:
Xtr_bot.shape

(4286, 23)

In [107]:
X_bot.shape

(4999, 23)

#### CFS

In [108]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# create the feature selector
selector = SelectKBest(score_func=mutual_info_classif, k=18)
# fit the feature selector on the data
selector.fit(Xtr_bot, ytr_bot)

# get the selected features
Xtr_bot = selector.transform(Xtr_bot)

X_bot = selector.transform(X_bot)

In [109]:
X_train, X_valid6, y_train, y_valid6 = train_test_split(Xtr_bot, ytr_bot, test_size=0.20, random_state=42)

In [110]:
Xtr_bot.shape

(4286, 18)

In [111]:
X_bot.shape

(4999, 18)

In [112]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

# create the model
model2 = Sequential()
model2.add(Dense(64, input_dim=18, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(16, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid'))

# compile the model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model2.fit(Xtr_bot,ytr_bot, epochs=69, batch_size=64)

Epoch 1/69
67/67 [==============================] - 1s 1ms/step - loss: 0.5055 - accuracy: 0.7788
Epoch 2/69
67/67 [==============================] - 0s 1ms/step - loss: 0.4238 - accuracy: 0.8161
Epoch 3/69
67/67 [==============================] - 0s 2ms/step - loss: 0.4115 - accuracy: 0.8297
Epoch 4/69
67/67 [==============================] - 0s 2ms/step - loss: 0.3967 - accuracy: 0.8292
Epoch 5/69
67/67 [==============================] - 0s 2ms/step - loss: 0.3922 - accuracy: 0.8315
Epoch 6/69
67/67 [==============================] - 0s 2ms/step - loss: 0.3971 - accuracy: 0.8299
Epoch 7/69
67/67 [==============================] - 0s 2ms/step - loss: 0.3926 - accuracy: 0.8301
Epoch 8/69
67/67 [==============================] - 0s 2ms/step - loss: 0.3837 - accuracy: 0.8339
Epoch 9/69
67/67 [==============================] - 0s 1ms/step - loss: 0.3781 - accuracy: 0.8362
Epoch 10/69
67/67 [==============================] - 0s 1ms/step - loss: 0.3827 - accuracy: 0.8381
Epoch 11/69
67/67 [

In [113]:
predictions_bot = model2.predict(X_bot)

157/157 [==============================] - 0s 832us/step


In [114]:
predictions_bot.shape

(4999, 1)

In [115]:
predictions_bot.sum()

1063.1752

In [116]:
max(predictions_bot)

array([0.9831219], dtype=float32)

In [117]:
min(predictions_bot)

array([6.609844e-25], dtype=float32)

In [118]:
modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
#modelPredUser

# PREPARE SUBMISSION

You will need to submit exact same file produced by using the following code. Any deviation from the desired format willbe marked as 0.

In [119]:
# Explain your approach

data_explanations = '''
To strengthen our training data, we concatenated all annotation files (from each group member) with the main training datasets.
Since the given prediction test dataset can not be verified (no labels provided), we divided our dataset into two subsets of 80% allocated for training and 20% set aside for testing, in order to evaluate the performance of our machine learning models.
Just Before the prediction, we trained our models with all of the labeled data we have and predicted to evaluation data.
'''

feature_explanations = '''
For feature extraction process, we discussed our annotation methodologies for both isPolitical and isBot parts.
Based on our common approaches, we selected the most useful features which were not included in the base template model.

created_at = the account's creation date.
since: it refers to the time passed from date that the account has been created 
user_geo_enabled : it is 1 if the user enabled his location.
user_verified : it refers to whether the account has verified by Twitter or not. 
count_digits : It refers to the amount of digits that exist in a username.
user_protected: It is 1 if the user has a private account.
user_followers_count: the number of followers that a user has.
user_friends_count: the number of people that the user is following.
description_len: the number of characters of the bio (description) of the user. 
user_tweet_count: the number of tweets that the user tweeted.
user_fav_count: the number of tweets that the user favorited.
average_tweet: the average amount of tweet that user tweeted since he joined to Twitter.
average_fav: the average amount of tweets that user favorited since he joined to Twitter.
followers_to_all_ratio: followers / (followers + friends)
is_reply: it indicates whether the tweet is a reply to another tweet.
replied_username: the username of the tweet which is replied.
tweet_length: the lenght of the tweet.
punctuation: the number of punctuation marks.

and additionally, we added more keywords to the political entities list.


Then, for round 2 we extracted more feature and at total isPolitical set has 32 and isBot set has 23 features.
We only used numerical and boolean variables. Numeric features are scaled with standardscaler.
Best 10 features are selected for political and 18 features are selected for bot.

'''

model_explanations = '''
In our modelling we had used following modelling methodologies and compared their Accuracy and MSE scores to determine best model.
GridSearch Cross Validation is applied to XGBoost, AdaBoost, DecisionTree to find best hyperparameter values.
Standard scaling is applied to all numerical values (normalization). In Round2, we tested voting classifier with
XGBoost, AdaBoost, LGBM, RandomForest and NN (Keras Binary Classifier) and obtained better results. However, we decided to
use NN directly since it can give us the highest MSE score due to its continuous outputs. We used NN for both
classification tasks. 

Here our tested Models 
    Decision Tree
    Random Forest
    Logistic Regression
    XGBoost 
    AdaBoost 
    LGBM 
    NLP-BERT 
    KNN (K-Nearest Neighbors)
    Stacking Classifier
    Soft Voting Classifier
    Bagging Classifier
    Gaussian Naive Bayes 
    CNN
    NN
    
We are no longer planning to utilize NPL-BERT model (NLP, encoded-base tranformer model-BERT) for isPolitical part,
and used CNN for the isBot and isPolitical part both.

'''

additional_explanations = '''
It seems that improving the quality and the quantity of the political entity list, increases the accuracy rate we have obtained significantly.
Also the adding the annotation files to the training set is something we have brain-stormed. We detected and generated many useful features
howerer as we add more and more feature to our training data, mse has increased. Therefore we decided to move on with bestfeatures
feature selection method. 
'''

In [120]:
ROUND = 3 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = '26772'#'<insert-your-id-here>'

In [121]:
predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}


with open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))

In [122]:
# Test your submission file

submission = json.load(open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'r'))
submission

{'round': 3,
 'student_id': '26772',
 'user_predictions': {'biologselim': 0.22826643288135529,
  'omerakdag34': 0.004544129129499197,
  'bilgin21604923': 0.21883893013000488,
  '_sydneycarton_': 0.03216167166829109,
  'denizlihabercom': 0.2072497308254242,
  'burakerbaychp': 0.005053309723734856,
  'mvnez': 0.025849323719739914,
  'qara118': 0.3034898340702057,
  'nabiyonyevrum': 0.0073475586250424385,
  'farukhalit2': 0.014434917829930782,
  'harlunoshi': 0.011533812619745731,
  'heritagepaix': 0.028760641813278198,
  'nuranwolf': 0.03573404997587204,
  'politikgundem': 0.14117524027824402,
  'isakethudax': 0.10498625785112381,
  'enveraysevera': 0.04688059166073799,
  'ilaydejaneiro': 0.01764584518969059,
  '1905anason': 0.3791353106498718,
  'eraydurgut03': 0.027661234140396118,
  'dasiskein': 0.07989341020584106,
  'ercan_bas29': 0.20631520450115204,
  'mett_1907': 0.039448998868465424,
  'ondemir066': 0.4009200632572174,
  'semihyeteer': 0.0157169159501791,
  'haberinyokcokk': 0.0

In [123]:
preds = model2.predict(X_valid6)

27/27 [==============================] - 0s 809us/step


In [124]:
# from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
mse = mean_squared_error(y_valid6, preds)
mse

0.10751988373651625

In [125]:
preds2 = model1.predict(X_valid5)

28/28 [==============================] - 0s 852us/step


In [126]:
# from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
mse = mean_squared_error(y_valid5, preds2)
mse

0.1454844862932966